# Example of text BOW
This is an example from sklearn (http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html) of using BOW for featurizing text.
Insert as second motivation for discussion featurization.

Also -- for this and other examples, as a concept check question have students explicitly write down/discuss what $\phi(X)$ is (i.e. $\phi(X) = [\#word_{1},...,\#word_{35788}]$ sparsified).

In [1]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']

In [2]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',
     categories=categories, shuffle=True, random_state=42)


In [3]:
len(twenty_train.data)

2257

In [4]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


In [5]:
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


## Bags of words
The most intuitive way to featurize text is the bags of words representation:

- assign a fixed integer id to each word occurring in any document of the training set (for instance by building a dictionary from words to integer indices).
- for each document $i$, count the number of occurrences of each word $w$ and store it in $X[i, j]$ as the value of feature $j$ where $j$ is the index of word $w$ in the dictionary
The bags of words representation implies that $n_{features}$ is the number of distinct words in the corpus: this number is typically larger than 100,000.

If $n_{samples}$ == 10000, storing $X$ as a numpy array of type float32 would require 10000 x 100000 x 4 bytes = 4GB in RAM.

Fortunately, most values in $X$ will be zeros since for a given document less than a couple thousands of distinct words will be used. For this reason we say that bags of words are typically high-dimensional sparse datasets. We can save a lot of memory by only storing the non-zero parts of the feature vectors in memory.

scipy.sparse matrices are data structures that do exactly this, and scikit-learn has built-in support for these structures.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

In [39]:
type(X_train_counts)

scipy.sparse.csr.csr_matrix

In [7]:
count_vect.vocabulary_.get(u'algorithm')

4690

## TF-IDF

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

## Build our classifier

In [9]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42,
                                           max_iter=5, tol=None)),
])
text_clf.fit(twenty_train.data, twenty_train.target)  

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False))])

In [11]:
twenty_test = fetch_20newsgroups(subset='test',
    categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data

In [13]:
import numpy as np

In [14]:
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)            

0.9127829560585885

In [15]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,
    target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.81      0.87       319
         comp.graphics       0.88      0.97      0.92       389
               sci.med       0.94      0.90      0.92       396
soc.religion.christian       0.90      0.95      0.93       398

           avg / total       0.92      0.91      0.91      1502



In [33]:
for doc, category in zip(twenty_test.data[3:6], predicted[3:6]):
     print('%s... => %s\n\n' % (doc[0:100], twenty_train.target_names[category]))

From: livesey@solntze.wpd.sgi.com (Jon Livesey)
Subject: Re: free moral agency
Organization: sgi
Lin... => alt.atheism


From: jhpb@sarto.budd-lake.nj.us (Joseph H. Buehler)
Subject: the ancient canon of the Roman rite
Or... => soc.religion.christian


From: bil@okcforum.osrhe.edu (Bill Conner)
Subject: Re: free moral agency
Nntp-Posting-Host: okcforu... => alt.atheism


